In [1]:
import fastf1
from fastf1 import _api as api
from fastf1.logger import (
    get_logger,
    soft_exceptions
)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

session = fastf1.get_session(2024, "Belgium", 'R')

req         WARNING 	DEFAULT CACHE ENABLED! (87.36 MB) C:\Users\EU10698\AppData\Local\Temp\fastf1
logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	Using cached data for season_schedule


In [3]:
_logger = get_logger('api')

EMPTY_LAPS = {'Time': pd.NaT, 'Driver': '', 'LapTime': pd.NaT,
              'NumberOfLaps': np.nan, 'NumberOfPitStops': np.nan,
              'PitOutTime': pd.NaT, 'PitInTime': pd.NaT,
              'Sector1Time': pd.NaT, 'Sector2Time': pd.NaT,
              'Sector3Time': pd.NaT, 'Sector1SessionTime': pd.NaT,
              'Sector2SessionTime': pd.NaT, 'Sector3SessionTime': pd.NaT,
              'SpeedI1': np.nan, 'SpeedI2': np.nan, 'SpeedFL': np.nan,
              'SpeedST': np.nan, 'IsPersonalBest': False}

EMPTY_STREAM = {'Time': pd.NaT, 'Driver': '', 'Position': np.nan,
                'GapToLeader': np.nan, 'IntervalToPositionAhead': np.nan}

In [4]:
def _extended_timing_data(path, response=None, livedata=None):
    # extended over the documentation of ``timing_data``:
    #   - returns session_split_times for splitting Q1/Q2/Q3 additionally
    # possible optional sanity checks (TODO, maybe):
    #   - inlap has to be followed by outlap
    #   - pit stops may never be negative (missing outlap)
    #   - speed traps against telemetry (especially in Q FastLap - Slow Lap)
    if livedata is not None and livedata.has('TimingData'):
        response = livedata.get('TimingData')
    elif response is None:  # no previous response provided
        _logger.info("Fetching timing data...")
        response = api.fetch_page(path, 'timing_data')
        if response is None:  # no response received
            raise SessionNotAvailableError(
                "No data for this session! If this session only finished "
                "recently, please try again in a few minutes."
            )
    _logger.info("Parsing timing data...")

    # split up response per driver for easier iteration and processing later
    resp_per_driver = dict()
    for entry in response:
        if (len(entry) < 2) or 'Lines' not in entry[1]:
            continue
        for drv in entry[1]['Lines']:
            if drv not in resp_per_driver.keys():
                resp_per_driver[drv] = [(entry[0], entry[1]['Lines'][drv])]
            else:
                resp_per_driver[drv].append((entry[0], entry[1]['Lines'][drv]))

    # create empty data dicts and populate them with data from all drivers after that
    laps_data = {key: list() for key, val in EMPTY_LAPS.items()}
    stream_data = {key: list() for key, val in EMPTY_STREAM.items()}

    session_split_times = [datetime.timedelta(days=1), ] * 3

    return resp_per_driver



resp_per_driver = _extended_timing_data(session.api_path, livedata=None)

1778511725       INFO 	Fetching timing data...
1778511725       INFO 	Parsing timing data...


In [24]:
from fastf1.utils import (
    recursive_dict_get,
    to_datetime,
    to_timedelta
)

def _laps_data_driver(driver_raw, empty_vals, drv):
    """
    .. warning::
        :mod:`fastf1.api` will be considered private in future releases and
        potentially be removed or changed.

    Data is on a per-lap basis.

    Boolean flag 'PitOut' is not evaluated. Meaning is unknown and flag is only sometimes present when a car leaves
    the pits.

    Params:
        driver_raw (list): raw api response for this driver only [(Timestamp, data), (...), ...]
        empty_vals (dict): dictionary of column names and empty column values
        drv (str): driver identifier

    Returns:
         dictionary of laps data for this driver
    """

    integrity_errors = list()

    # do a quick first pass over the data to find out when laps start and end
    # this is needed so we can work with a more efficient "look ahead" on the main pass
    # example: we can have 'PitOut' 0.01s before a new lap starts, but 'PitOut' belongs to the new lap, not the old one

    lapcnt = 0  # we're keeping two separate lap counts because sometimes the api has a non existent lap too much...
    api_lapcnt = 0  # ...at the beginning; we can correct that though;
    # api_lapcnt does not count backwards even if the source data does
    in_past = False  # flag for when the data went back in time
    out_of_pit = False  # flag set to true when driver drives out FOR THE FIRST TIME; stays true from then on

    # entries are prefilled with empty values and only overwritten if they exist in the response line
    drv_data = {key: [val, ] for key, val in empty_vals.items()}

    for time, resp in driver_raw:
        # the first three ifs are just edge case handling for the rare sessions were the data goes back in time
        if in_past and 'NumberOfLaps' in resp and resp['NumberOfLaps'] == api_lapcnt:
            in_past = False  # we're back in the present

        if 'NumberOfLaps' in resp and ((prev_lapcnt := resp['NumberOfLaps']) < api_lapcnt):
            _logger.warning(f"Driver {drv: >2}: Ignoring late data for a "
                            f"previously processed lap.The data may contain "
                            f"errors (previous: {prev_lapcnt}; "
                            f"current {lapcnt})")
            in_past = True
            continue

        if in_past:  # still in the past, just continue and ignore everything
            continue

        if ('InPit' in resp) and (resp['InPit'] is False):
            out_of_pit = True  # drove out of the pits for the first time

        # new lap; create next row
        if 'NumberOfLaps' in resp and resp['NumberOfLaps'] > api_lapcnt:
            api_lapcnt += 1
            # make sure the car actually drove out of the pits already; it can't be a new lap if it didn't
            if out_of_pit:
                drv_data['Time'][lapcnt] = to_timedelta(time)
                lapcnt += 1
                # append a new empty row; last row may not be populated (depending on session) and may be removed later
                for key, val in empty_vals.items():
                    drv_data[key].append(val)

    # now, do the main pass where all the other data is actually filled in
    # same counters and flags as before, reset them
    lapcnt = 0  # we're keeping two separate lap counts because sometimes the api has a non existent lap too much...
    api_lapcnt = 0  # ...at the beginning; we can correct that though;
    # api_lapcnt does not count backwards even if the source data does
    in_past = False  # flag for when the data went back in time

    personal_best_lap_times = list()

    session_split_times = [datetime.timedelta(0)]
    # start times of (sub)sessions (Q1, Q2, Q3)

    pitstops = -1  # start with -1 because first is out lap, needs to be zero after that

    # iterate through the data; new lap triggers next row in data
    for time, resp in driver_raw:
        # the first three ifs are just edge case handling for the rare sessions were the data goes back in time
        if in_past and 'NumberOfLaps' in resp and resp['NumberOfLaps'] == api_lapcnt:
            in_past = False  # we're back in the present
        if in_past or ('NumberOfLaps' in resp and resp['NumberOfLaps'] < api_lapcnt):
            in_past = True
            continue

        # values which are up to five seconds late are still counted towards the previous lap
        # (sector times, speed traps and lap times)
        lap_offset = 0
        if (lapcnt > 0) and (to_timedelta(time) - drv_data['Time'][lapcnt - 1] < pd.Timedelta(5, 's')):
            lap_offset = 1

        if 'Sectors' in resp and isinstance(resp['Sectors'], dict):
            # sometimes it's a list but then it never contains values...
            for sn, sector, sesst in (('0', 'Sector1Time', 'Sector1SessionTime'),
                                      ('1', 'Sector2Time', 'Sector2SessionTime'),
                                      ('2', 'Sector3Time', 'Sector3SessionTime')):
                if val := recursive_dict_get(resp, 'Sectors', sn, 'Value'):
                    drv_data[sector][lapcnt - lap_offset] = to_timedelta(val)
                    drv_data[sesst][lapcnt - lap_offset] = to_timedelta(time)

        if val := recursive_dict_get(resp, 'LastLapTime', 'Value'):
            # if 'LastLapTime' is received less than five seconds after the start of a new lap, it is still added
            # to the last lap
            val = to_timedelta(val)
            if val.total_seconds() < 150:
                # laps which are longer than 150 seconds are ignored; usually this is the case between Q1, Q2 and Q3
                # because all three qualifying sessions are one session here. Those timestamps are often wrong and
                # sometimes associated with the wrong lap
                drv_data['LapTime'][lapcnt - lap_offset] = val

        if 'Speeds' in resp:
            for trapkey, trapname in (('I1', 'SpeedI1'), ('I2', 'SpeedI2'), ('FL', 'SpeedFL'), ('ST', 'SpeedST')):
                if val := recursive_dict_get(resp, 'Speeds', trapkey, 'Value'):
                    # speed has to be float because int does not support NaN
                    if trapkey == 'ST':
                        # the ST trap value can occur early enough in a new lap
                        # that it needs to be excluded from the usual offset
                        # logic, therefore the offset is ignored here
                        drv_data[trapname][lapcnt] = float(val)
                    else:
                        drv_data[trapname][lapcnt - lap_offset] = float(val)

        if 'InPit' in resp:
            # 'InPit': True is received once when entering pits, False is received once when leaving
            if resp['InPit'] is True:
                if pitstops >= 0:
                    drv_data['PitInTime'][lapcnt] = to_timedelta(time)
            elif ((('NumberOfLaps' in resp) and resp['NumberOfLaps'] > api_lapcnt)
                  or (drv_data['Time'][lapcnt] - to_timedelta(time))
                  < pd.Timedelta(5, 's')):
                # same response line as beginning of next lap
                # or beginning of next lap less than 5 seconds away
                drv_data['PitOutTime'][lapcnt + 1] = to_timedelta(time)  # add to next lap
                pitstops += 1
            else:
                drv_data['PitOutTime'][lapcnt] = to_timedelta(time)  # add to current lap
                pitstops += 1

        # Get save information about personal best lap times at the timestamp
        # at which this information was received.
        # Whenever a lap is deleted (if that happens quickly after it was set),
        # the previous 'BestLapTime' value is sent again. There is some extra
        # logic at then end that correctly marks personal best laps based on
        # the data that is saved here.
        if val := recursive_dict_get(resp, 'BestLapTime', 'Value'):
            personal_best_lap_times.append(
                (to_timedelta(time), to_timedelta(val))
            )

        # Create approximate (sub)session (i.e. quali) split times by
        # (mis)using the session number counter from 'BestLapTimes'.
        # (Note: those lap times cannot be used for correct personal best
        #  detection, because the previous value is not resent here when a lap
        #  is deleted.)
        if (val := resp.get('BestLapTimes')) and isinstance(val, dict):
            session_n = int(list(val.keys())[0])
            if (session_n + 1) > len(session_split_times):
                session_split_times.append(to_timedelta(time))

        # new lap; create next row
        if 'NumberOfLaps' in resp and resp['NumberOfLaps'] > api_lapcnt:
            api_lapcnt += 1
            # make sure the car actually drove out of the pits already; it can't be a new lap if it didn't
            if pitstops >= 0:
                drv_data['Time'][lapcnt] = to_timedelta(time)
                drv_data['NumberOfLaps'][lapcnt] = lapcnt + 1  # don't use F1's lap count; ours is better
                drv_data['NumberOfPitStops'][lapcnt] = pitstops
                drv_data['Driver'][lapcnt] = drv
                lapcnt += 1

    if lapcnt == 0:  # no data at all for this driver
        return None, None

    # done reading the data, do postprocessing

    def data_in_lap(lap_n):
        relevant = ('Sector1Time', 'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2',
                    'SpeedFL', 'SpeedST', 'LapTime')
        for col in relevant:
            if not pd.isnull(drv_data[col][lap_n]):
                return True
        return False

    # 'NumberOfLaps' always introduces a new lap (can be a previous one) but sometimes there is one more lap at the end
    # in this case the data will be added as usual above, lap count and pit stops are added here and the 'Time' is
    # calculated below from sector times
    if data_in_lap(lapcnt):
        drv_data['NumberOfLaps'][lapcnt] = lapcnt + 1
        drv_data['NumberOfPitStops'][lapcnt] = pitstops
        drv_data['Driver'][lapcnt] = drv
    else:  # if there was no more data after the last lap count increase,
        # delete the last empty record
        for key in drv_data.keys():
            drv_data[key] = drv_data[key][:-1]
    if not data_in_lap(0):  # remove first lap if there's no data;
        # "pseudo outlap" that didn't exist
        for key in drv_data.keys():
            drv_data[key] = drv_data[key][1:]
        drv_data['NumberOfLaps'] = list(map(lambda n: n - 1, drv_data['NumberOfLaps']))  # reduce each lap count by one

    if not drv_data['Time']:
        # ensure that there is still data left after potentially removing a lap
        return drv_data, session_split_times

    for i in range(len(drv_data['Time'])):
        sector_sum = datetime.timedelta(0)
        na_sectors = list()  # list of keys for missing sector times
        for key in ('Sector1Time', 'Sector2Time', 'Sector3Time'):
            st = drv_data[key][i]
            if pd.isna(st):
                na_sectors.append(key)
                continue
            sector_sum += st

        # check for incorrect lap times and remove them
        # fixes GH#167 among others
        if sector_sum > drv_data['LapTime'][i]:
            drv_data['LapTime'][i] = pd.NaT
            integrity_errors.append(i + 1)

        if i == 0:
            # only do following corrections for 2nd lap and onwards
            continue

        # The API only sends and update if a state changes, therefore, if two
        # lap times or sector times are exactly equal, the second value will
        # be missing. Missing sector times and lap times are calculated here
        # based on the available values for a lap (max one may be missing). If
        # the calculated value matches the previous value, it will be set.

        # lap time is missing
        if (not na_sectors) and pd.isna(drv_data['LapTime'][i]) \
                and (drv_data['LapTime'][i - 1] == sector_sum):

            drv_data['LapTime'][i] = sector_sum

        # one sector time is missing
        elif (len(na_sectors) == 1) and not pd.isna(drv_data['LapTime'][i]):
            # create a list with the two keys for available sector times
            ref_sec = ['Sector1Time', 'Sector2Time', 'Sector3Time']
            ref_sec.remove(na_sectors[0])

            if (sec1 := (drv_data['LapTime'][i]
                         - drv_data[ref_sec[0]][i]
                         - drv_data[ref_sec[1]][i])) \
                    == drv_data[na_sectors[0]][i - 1]:

                drv_data[na_sectors[0]][i] = sec1

    # lap time sync; check which sector time was triggered with the lowest latency
    # Sector3SessionTime == end of lap
    # Sector2SessionTime + Sector3Time == end of lap
    # Sector1SessionTime + Sector2Time + Sector3Time == end of lap
    # all of these three have slightly different times; take earliest one -> most exact because can't trigger too early
    for i in range(len(drv_data['Time'])):
        sector_sum = pd.Timedelta(0)
        min_time = drv_data['Time'][i]
        for sector_time, session_time in ((pd.Timedelta(0), drv_data['Sector3SessionTime'][i]),
                                          (drv_data['Sector3Time'][i], drv_data['Sector2SessionTime'][i]),
                                          (drv_data['Sector2Time'][i], drv_data['Sector1SessionTime'][i])):
            if pd.isnull(session_time):
                continue
            if pd.isnull(sector_time):
                break  # need to stop here because else the sector sum will be incorrect

            sector_sum += sector_time
            new_time = session_time + sector_sum
            print(session_time, "|", sector_sum, "|", sector_time)
            if not pd.isnull(new_time) and (new_time < min_time or pd.isnull(min_time)):
                min_time = new_time
        if i > 0 and min_time < drv_data['Time'][i - 1]:
            integrity_errors.append(i + 1)  # not be possible if sector times and lap time are correct
            continue

        drv_data['Time'][i] = min_time
        print(i, drv_data['Time'][i], new_time)
        print("==============================================")

    # last lap needs to be removed if it does not have a 'Time' and it could not be calculated (likely an inlap)
    if pd.isnull(drv_data['Time'][-1]):
        if not pd.isnull(drv_data['PitInTime'][-1]):
            drv_data['Time'][-1] = drv_data['PitInTime'][-1]
        else:
            for key in drv_data.keys():
                drv_data[key] = drv_data[key][:-1]

    if not drv_data['Time']:
        # ensure that there is still data left after potentially removing a lap
        return drv_data, session_split_times

    # more lap sync, this time check which lap triggered with the lowest latency
    for i in range(len(drv_data['Time']) - 1, 0, -1):
        print(i)
        if (new_time := drv_data['Time'][i] - drv_data['LapTime'][i]) < \
                drv_data['Time'][i - 1]:
            if i > 1 and new_time < drv_data['Time'][i - 2]:
                integrity_errors.append(i + 1)  # not be possible if sector times and lap time are correct
            else:
                print("------------------------------")
                print(drv_data['Time'][i]," - ", drv_data['LapTime'][i])
                print(drv_data['Time'][i] - drv_data['LapTime'][i])
                print(i-1, drv_data['Time'][i - 1], " >>>> ", new_time)
                drv_data['Time'][i - 1] = new_time

    # need to go both directions once to make everything match up; also recalculate sector times
    for i in range(len(drv_data['Time']) - 1):
        if any(pd.isnull(tst) for tst in (
                drv_data['Time'][i], drv_data['LapTime'][i + 1],
                drv_data['Sector1Time'][i + 1],
                drv_data['Sector2Time'][i + 1],
                drv_data['Sector3Time'][i + 1])):
            continue  # lap not usable, missing critical values
        
        print(drv_data['Time'][i+1])
        if (new_time := drv_data['Time'][i] + drv_data['LapTime'][i+1]) \
                < drv_data['Time'][i+1]:
            drv_data['Time'][i+1] = new_time
            # print(i+1, new_time)

        if (new_s1_time := drv_data['Time'][i]
                + drv_data['Sector1Time'][i+1]) \
                < drv_data['Sector1SessionTime'][i+1]:
            drv_data['Sector1SessionTime'][i+1] = new_s1_time
        if (new_s2_time := drv_data['Time'][i] + drv_data['Sector1Time'][i+1]
                + drv_data['Sector2Time'][i+1]) \
                < drv_data['Sector2SessionTime'][i+1]:
            drv_data['Sector2SessionTime'][i+1] = new_s2_time
        if (new_s3_time := drv_data['Time'][i] + drv_data['Sector1Time'][i+1]
                + drv_data['Sector2Time'][i+1]
                + drv_data['Sector3Time'][i+1]) \
                < drv_data['Sector3SessionTime'][i+1]:
            drv_data['Sector3SessionTime'][i+1] = new_s3_time

    # Iterate over list of personal lap times set 'IsPersonalBest'.
    # When a lap is deleted, the API resends the previous personal best.
    # Therefore, by iterating in reverse, if any lap is encountered that is
    # quicker than already processed personal best lap times, it must have
    # been deleted.
    # This is just best effort but not exhaustive as it can only handle lap
    # times that were deleted quickly (before the next personal best was set).
    _corrected_personal_best_lap_times = list()
    # list is only used for backreference within the loop
    cur_sn = len(session_split_times) - 1
    # current (sub)session number, personal best lap times need to be
    # considered for each (sub)session individually
    for time, pb_lap_time in reversed(personal_best_lap_times):
        if time < session_split_times[cur_sn]:
            # transitioned into the previous (sub)session (reverse iteration!)
            # reset the reference list, so time are considered individually
            cur_sn -= 1
            _corrected_personal_best_lap_times = list()

        if _corrected_personal_best_lap_times:
            if pb_lap_time in _corrected_personal_best_lap_times:
                continue
            elif pb_lap_time < min(_corrected_personal_best_lap_times):
                continue

        _corrected_personal_best_lap_times.append(pb_lap_time)

        # find the index of the corresponding lap by comparing with the lap
        # times and set 'IsPersonalBest' to True for that lap
        try:
            pb_idx = drv_data['LapTime'].index(pb_lap_time)
        except ValueError:
            # one example case where this error occurs, are wildly of personal
            # best times (>2 min lap time) that are sometimes present and
            # which have no corresponding lap time
            pass
        else:
            drv_data['IsPersonalBest'][pb_idx] = True

    # fix the number of pit stops; due to potentially multiple laps to the grid
    # where a car goes through the pit lane before finally taking its place
    # on the grid, the number of pit stops on the first lap may be already
    # greater than zero; therefore, apply correction so that we start with zero
    pitstop_offset = drv_data['NumberOfPitStops'][0]
    for i in range(len(drv_data['NumberOfPitStops'])):
        drv_data['NumberOfPitStops'][i] -= pitstop_offset

    # fix first lap PitInTime; same reason as above for pit stops, there may
    # be an incorrect PitInTime on the first lap. There always is a PitOutTime
    # for when the car leaves the box for the lap to the grid. There is a
    # PitInTime if the car drives multiple laps to the grid, discard these.
    # There is also a PitInTime if the car actually pits at the end of the
    # first lap, those need to be kept.
    if drv_data['PitInTime'][0] < drv_data['PitOutTime'][0]:
        drv_data['PitInTime'][0] = pd.NaT

    if integrity_errors:
        _logger.warning(
            f"Driver {drv: >2}: Encountered {len(integrity_errors)} timing "
            f"integrity error(s) near lap(s): {integrity_errors}.\n"
            f"This might be a bug and should be reported.")

    return drv_data, session_split_times


# for drv in resp_per_driver.keys():
#     drv_laps_data, drv_session_split_times \
#         = _laps_data_driver(resp_per_driver[drv], EMPTY_LAPS, drv)
#     print(drv)
#     print(drv_laps_data)
#     break

drv = "11"
drv_laps_data, drv_session_split_times \
    = _laps_data_driver(resp_per_driver[drv], EMPTY_LAPS, drv)
pd.DataFrame(drv_laps_data)


0:57:08.631000 | 0 days 00:00:00 | 0 days 00:00:00
0:56:39.019000 | 0 days 00:00:29.491000 | 0:00:29.491000
0 0 days 00:57:08.510000 0 days 00:57:08.510000
0:58:58.806000 | 0 days 00:00:00 | 0 days 00:00:00
0:58:29.199000 | 0 days 00:00:29.668000 | 0:00:29.668000
0:57:39.952000 | 0 days 00:01:18.875000 | 0:00:49.207000
1 0:58:58.806000 0 days 00:58:58.827000
1:00:48.902000 | 0 days 00:00:00 | 0 days 00:00:00
1:00:19.226000 | 0 days 00:00:29.600000 | 0:00:29.600000
0:59:30.427000 | 0 days 00:01:18.513000 | 0:00:48.913000
2 0 days 01:00:48.826000 0 days 01:00:48.940000
1:02:38.701000 | 0 days 00:00:00 | 0 days 00:00:00
1:02:09.175000 | 0 days 00:00:29.402000 | 0:00:29.402000
1:01:20.304000 | 0 days 00:01:18.434000 | 0:00:49.032000
3 0 days 01:02:38.577000 0 days 01:02:38.738000
1:04:28.501000 | 0 days 00:00:00 | 0 days 00:00:00
1:03:59.032000 | 0 days 00:00:29.491000 | 0:00:29.491000
1:03:10.126000 | 0 days 00:01:18.449000 | 0:00:48.958000
4 1:04:28.501000 0 days 01:04:28.575000
1:06:18.

,Time,Driver,LapTime,NumberOfLaps,NumberOfPitStops,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest
0,0 days 00:57:08.507000,11,NaT,1,0,0 days 00:19:13.289000,NaT,NaT,0 days 00:00:50.038000,0 days 00:00:29.491000,NaT,0 days 00:56:39.019000,0 days 00:57:08.631000,316.0,201.0,216.0,310.0,False
1,0 days 00:58:58.747000,11,0 days 00:01:50.240000,2,0,NaT,NaT,0 days 00:00:31.365000,0 days 00:00:49.207000,0 days 00:00:29.668000,0 days 00:57:39.872000,0 days 00:58:29.079000,0 days 00:58:58.747000,325.0,202.0,217.0,308.0,True
2,0 days 01:00:48.730000,11,0 days 00:01:49.983000,3,0,NaT,NaT,0 days 00:00:31.470000,0 days 00:00:48.913000,0 days 00:00:29.600000,0 days 00:59:30.217000,0 days 01:00:19.130000,0 days 01:00:48.730000,314.0,201.0,218.0,309.0,True
3,0 days 01:02:38.520000,11,0 days 00:01:49.790000,4,0,NaT,NaT,0 days 00:00:31.356000,0 days 00:00:49.032000,0 days 00:00:29.402000,0 days 01:01:20.086000,0 days 01:02:09.118000,0 days 01:02:38.520000,320.0,199.0,218.0,NaN,True
4,0 days 01:04:28.416000,11,0 days 00:01:49.896000,5,0,NaT,NaT,0 days 00:00:31.447000,0 days 00:00:48.958000,0 days 00:00:29.491000,0 days 01:03:09.967000,0 days 01:03:58.925000,0 days 01:04:28.416000,319.0,197.0,215.0,311.0,False
5,0 days 01:06:18.545000,11,0 days 00:01:50.129000,6,0,NaT,NaT,0 days 00:00:31.583000,0 days 00:00:49.116000,0 days 00:00:29.430000,0 days 01:04:59.999000,0 days 01:05:49.115000,0 days 01:06:18.545000,312.0,198.0,215.0,NaN,False
6,0 days 01:08:08.182000,11,0 days 00:01:49.637000,7,0,NaT,NaT,0 days 00:00:31.524000,0 days 00:00:48.599000,0 days 00:00:29.514000,0 days 01:06:50.069000,0 days 01:07:38.668000,0 days 01:08:08.182000,321.0,200.0,215.0,309.0,True
7,0 days 01:09:58.094000,11,0 days 00:01:49.912000,8,0,NaT,NaT,0 days 00:00:31.530000,0 days 00:00:48.828000,0 days 00:00:29.554000,0 days 01:08:39.712000,0 days 01:09:28.540000,0 days 01:09:58.094000,314.0,199.0,216.0,310.0,False
8,0 days 01:11:48.380000,11,0 days 00:01:50.286000,9,0,NaT,NaT,0 days 00:00:31.506000,0 days 00:00:49.173000,0 days 00:00:29.607000,0 days 01:10:29.600000,0 days 01:11:18.773000,0 days 01:11:48.380000,315.0,199.0,216.0,NaN,False
9,0 days 01:13:38.437000,11,0 days 00:01:50.057000,10,0,NaT,NaT,0 days 00:00:31.598000,0 days 00:00:48.943000,0 days 00:00:29.516000,0 days 01:12:19.978000,0 days 01:13:08.921000,0 days 01:13:38.437000,314.0,198.0,216.0,309.0,False
